In [ ]:
# --- 1. インポート (重複をすべて削除し、クリーンアップ) ---
import numpy as np
import pandas as pd
import os
import time
import re
import logging
from time import perf_counter
from typing import Dict, Any
from pathlib import Path
from datetime import datetime

from ase.io import read, write, Trajectory
from ase.constraints import FixAtoms
from ase import units
from ase.build import add_vacuum

from sklearn.linear_model import LinearRegression

# --- Matlantis (PFP) のインポート ---
try:
    from pfp_api_client.pfp.estimator import Estimator, EstimatorCalcMode
    from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
    from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator as PFP # v7.0.0以降の推奨
    
    from matlantis_features.atoms import MatlantisAtoms
    from matlantis_features.utils.calculators import get_calculator, pfp_estimator_fn
    from matlantis_features.utils.atoms_util import convert_atoms_to_upper
    
    from matlantis_features.features.common.opt import FireLBFGSASEOptFeature
    
    # MDシミュレーションに必要なインポート
    from matlantis_features.features.md import (
        ASEMDSystem, 
        MDFeature, 
        MDExtensionBase,
        NVTBerendsenIntegrator, 
        NPTIntegrator, 
        NPTBerendsenIntegrator,
        LangevinIntegrator,
        VelocityVerletIntegrator # select_integratorで使う場合
    )
    from matlantis_features.features.md.md_extensions import DeformScheduler

except ImportError as e:
    print(f"Error: 必要なライブラリが見つかりません。- {e}")
    print("pfp-api-client (pfp-ase) と matlantis_features が正しくインストールされているか確認してください。")
    # エラーが発生した場合、ここで処理を停止させる
    raise e

# --- 2. MDヘルパー関数定義 ---

class PrintWriteLog(MDExtensionBase):
    """
    MDシミュレーションのログを記録・出力するクラス
    (定義は1回だけにします)
    """
    def __init__(self, fname: str, dirout: str = '.', stdout: bool = False):
        self.fname = fname
        self.dirout = dirout
        self.t_start = perf_counter()
        self.stdout = stdout

    def __call__(self, system, integrator):
        n_step = system.current_total_step
        sim_time = system.current_total_time / 1000    # ps
        E_tot = system.ase_atoms.get_total_energy()
        E_pot = system.ase_atoms.get_potential_energy()
        E_kin = system.ase_atoms.get_kinetic_energy()
        temp = system.ase_atoms.get_temperature()
        
        try:
            density = system.ase_atoms.get_masses().sum() / units.mol / (
                system.ase_atoms.cell.volume * (1e-8**3)
            )
        except Exception:
            density = 0.0 # 体積が0の場合などのエラー回避

        calc_time = (perf_counter() - self.t_start) / 60.  # min.

        # ヘッダー書き込み
        if n_step == 0:
            hdr = 'step,time[ps],E_tot[eV],E_pot[eV],E_kin[eV],'
            hdr += 'T[K],density[g/cm3],calc_time[min]'
            with open(f'{self.dirout}/{self.fname}.log', 'w') as f_log:
                f_log.write(f'{hdr}\n')

        # 結果の書き込みと出力
        line = f'{n_step:8d},{sim_time:7.2f},'
        line += f'{E_tot:11.4f},{E_pot:11.4f},{E_kin:9.4f},'
        line += f'{temp:8.2f},{density:7.3f},{calc_time:8.2f}'
        with open(f'{self.dirout}/{self.fname}.log', 'a') as f_log:
            f_log.write(f'{line}\n')
        if self.stdout:
            print(line)

# (注: select_integrator は元のコードでコメントアウトされていたため、ここでも除外します)

def run_md_simulation(
    atoms,
    integrator_type: str, # この引数は使われませんが、互換性のために残します
    temperature: float,
    n_steps: int,
    timestep: float = 1.0,
    pressure: float = 101325.0, # この引数も使われません
    traj_file: str = "md.traj",
    traj_freq: int = 50,
    logger_interval: int = 50,
    model_version: str = 'v7.0.0',
    calc_mode: str = 'CRYSTAL_U0',
    show_progress: bool = True,
    show_logger: bool = True,
):
    """
    MDシミュレーションを実行する汎用関数
    (定義は1回だけにします)
    (注: 元のコードに基づき、IntegratorはNVT_Berendsenに固定されています)
    """
    logger = logging.getLogger("matlantis_features")
    logger.setLevel(logging.INFO)
    estimator_fn = pfp_estimator_fn(model_version=model_version, calc_mode=calc_mode)
    
    # ASEMDSystemに渡すatomsがMatlantisAtomsであることを確認
    if not isinstance(atoms, MatlantisAtoms):
         atoms = MatlantisAtoms(atoms)
         
    system = ASEMDSystem(atoms)
    
    system.init_temperature(
        temperature=temperature,
        stationary=True,
        rng=np.random.RandomState(seed=12345)
    )
    print(f"システムを {temperature} K に初期化しました。")

    # 元のコードのロジックに基づき、NVT_Berendsenに固定
    integrator=NVTBerendsenIntegrator(
            timestep=timestep,
            temperature=temperature,
            taut=100.0 * units.fs
        )
    
    md = MDFeature(
        integrator=integrator,
        n_run=n_steps,
        show_progress_bar=show_progress,
        show_logger=show_logger,
        logger_interval=logger_interval,
        traj_file_name=traj_file,
        traj_freq=traj_freq,
        estimator_fn=estimator_fn,
    )
    
    print(f"\n--- MDシミュレーションを開始します ({n_steps} ステップ) ---")
    
    # ★★★ ここでシミュレーションが実行されます ★★★
    md_results = md(system)
    # ★★★★★★★★★★★★★★★★★★★★★★★★★★★★★
    
    print("\n--- MDシミュレーションが正常に完了しました ---")
    print(f"最終的なトラジェクトリは '{traj_file}' に保存されました。")
    print(f"最終ステップ: {system.current_total_step}, 最終時間: {system.current_time:.2f} fs")
    
    return md_results

# --- 3. メモリ解放 (計算実行前) ---
import gc
print("シミュレーション実行前にメモリを解放します...")
# 前のセルで定義された `optimized_atoms` などの巨大な変数を削除
# (もし `atoms` 変数にコピー済みで、`optimized_atoms` が不要な場合)
#
# try:
#     del optimized_atoms 
# except NameError:
#     pass # 変数が存在しない場合は何もしない

gc.collect() # ガベージコレクションを実行
print("メモリ解放完了。")


# --- 4. シミュレーションパラメータ設定 (重複を削除) ---

# (注: `INPUT_FILE` や `RELAX_TRAJ` など、このセルで使わないパラメータは削除しました)
# (注: `atoms` 変数は、前のセルで NPT緩和後の構造として定義されている前提です)

OUTPUT_TRAJ = 'PVDF_md_1600K.traj' # 出力トラジェクトリ

# --- 本番MD用のパラメータ ---
TARGET_TEMP_K = 1600.0      # 論文の図21の条件 (2000 K)
TIME_STEP_FS = 0.1          # 論文の条件 (0.1 fs)
TOTAL_TIME_PS = 40.0       # 論文の条件 (100 ps)

# --- 5. シミュレーション実行 ---

total_steps = int((TOTAL_TIME_PS * 1000) / TIME_STEP_FS) # 1,000,000 ステップ

# ログとトラジェクトリへの書き込み頻度 (ステップ数)
# 1000ステップごと (0.1 fs * 1000 = 100 fs = 0.1 psごと)
log_interval = 1000

print(f"\nStarting NVT MD simulation at {TARGET_TEMP_K} K...")
print(f"Time step: {TIME_STEP_FS} fs")
print(f"Total steps: {total_steps} (for {TOTAL_TIME_PS} ps)")
print(f"Log/Traj interval: {log_interval} steps")
# atoms=read("Al-Al2O3-Graphene-23PVDF.xyz")
atoms=read("/home/jovyan/Kaori/MD/LiB_2/structure/PVDF_only_shrunk.cif")
# atoms.set_cell([17.18,17.18,80.00])
pos=atoms.get_positions()
cell=[pos[:,0].max(),pos[:,1].max(),pos[:,2].max()]
atoms.set_cell(cell)
atoms.set_pbc([True,True,True])
fix_z_threshold = 6.0
fix_indices = [atom.index for atom in atoms if atom.position[2] < fix_z_threshold]

if fix_indices:
    print(f"z < {fix_z_threshold} Å にある {len(fix_indices)} 個の原子を固定します。")
    constraint = FixAtoms(indices=fix_indices)
    atoms.set_constraint(constraint)
else:
    print(f"z < {fix_z_threshold} Å にある原子は見つかりませんでした。制約は設定されません。")
# print(atoms.cell)
# `atoms` 変数がこのセルの前に定義されていることを確認
if 'atoms' not in locals():
    print("エラー: `atoms` 変数が定義されていません。")
    print("前のセルで NPT 緩和を実行し、その結果を `atoms` に代入してください。")
    # raise NameError("`atoms` is not defined.")
else:
    # メインのMDシミュレーションを実行
    md_results = run_md_simulation(
        atoms=atoms, # NPT緩和後の構造を使用
        integrator_type="NVT_Berendsen", # (この引数は関数内で上書きされます)
        temperature=TARGET_TEMP_K,
        n_steps=total_steps,
        timestep=TIME_STEP_FS,
        traj_file=OUTPUT_TRAJ,
        traj_freq=log_interval,
        logger_interval=log_interval,
        model_version='v7.0.0',      # MDに使用するモデル
        calc_mode='CRYSTAL_U0',      # MDに適した計算モード
        show_progress=True,
        show_logger=True,
    )

    print(f"\nMD simulation finished.")
    print(f"Trajectory saved to: {OUTPUT_TRAJ}")

    # --- 6. ★最重要★ メモリの即時解放 ---
    print("シミュレーションが完了しました。メモリを圧迫する可能性があるため、")
    print("結果変数 `md_results` を削除し、メモリを強制解放します...")
    
    try:
        del md_results # トラジェクトリファイルさえあれば、これは不要な場合が多い
        gc.collect()
        print("メモリ解放が完了しました。")
    except NameError:
        print("`md_results` が見つかりませんでしたが、処理は完了しています。")



シミュレーション実行前にメモリを解放します...
メモリ解放完了。

Starting NVT MD simulation at 1600.0 K...
Time step: 0.1 fs
Total steps: 400000 (for 40.0 ps)
Log/Traj interval: 1000 steps
z < 6.0 Å にある 204 個の原子を固定します。
システムを 1600.0 K に初期化しました。

--- MDシミュレーションを開始します (400000 ステップ) ---


  0%|          | 0/400000 [00:00<?, ?it/s]

The MD trajectory will be saved at /home/jovyan/Kaori/MD/LiB_2/structure/PVDF_md_1600K.traj.
Note: The max disk size of /home/jovyan is about 98G.
steps:     0  energy：-4.12 eV/atom  total energy: -3.95 eV/atom  temperature: 1593.43 K  volume: 32477 Ang^3  density: 0.770 g/cm^3
